In [110]:
!pip install pypng
!pip install pydicom
!pip install opencv-python
import numpy as np
import png
import pydicom
import cv2
import matplotlib.pylab as plt
import os
import shutil
import random

In [6]:
def read_png_and_contour_draw(png_name, contour_list):
    img = cv2.imread(png_name)
    color = (255, 0, 0)
    thickness = 2
    isClosed = True
    img = cv2.polylines(img, [contour_list], isClosed, color, thickness)
    plt.imshow(img)
    plt.show()

In [7]:
def create_mask_from_contour_list(cnt_file, path):
    contour_list = []
    with open(cnt_file) as f:
        lines = f.readlines()
        for lin in lines:
            cntrs = lin.strip().split(" ")
            contour_list.append([int(float(cntrs[0])), int(float(cntrs[1]))])
    mask = np.zeros((256, 256, 1), dtype = "uint8")
    pts = np.array(contour_list).reshape((-1, 1, 2))
    mask = cv2.fillPoly(mask, [pts], 255)
    cv2.imwrite(path + "mask/img/" + str(len(os.listdir(path + "mask/img/")) + 1) + ".png", mask)

In [11]:
def dicom_to_png_convert(name, path):
    ds = pydicom.dcmread(name)
    shape = ds.pixel_array.shape
    # Convert to float to avoid overflow or underflow losses.
    image_2d = ds.pixel_array.astype(float)
    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)
    # Write the PNG file
    with open(path + "images/img/" + str(len(os.listdir(path + "images/img/")) + 1) + ".png", 'wb') as png_file:
        w = png.Writer(shape[1], shape[0], greyscale=True)
        w.write(png_file, image_2d_scaled)

In [12]:
def create_training_testing_data():
    folder_no = 0
    base_folder = "Dataset"
    for part in os.listdir(base_folder):
        if "Contours" in part:
            level1 = base_folder + "/" + part
            level2 = level1 + "/" + os.listdir(level1)[0]
            for i in os.listdir(level2):
                if "Contours" in i:
                    level3 = level2 + "/" + i
                    print(folder_no, level3)
                    for j in os.listdir(level3):
                        if j[0:2] == "SC":
                            level4 = level3 + "/" + j
                            all_contours = os.listdir(level4 + "/contours-manual/IRCCI-expert")
                            image_path = level4.replace("Contours","DICOM") + "/DICOM"
                            for k in all_contours:
                                if "icontour" in k:
                                    final_img_pth = image_path + "/" + k[0:12] + ".dcm"
                                    final_mask_pth = level4 + "/contours-manual/IRCCI-expert/" + k
                                    path = 'Processed_Dataset/training/'
                                    if folder_no%2 == 1:
                                        path = 'Processed_Dataset/testing/'
                                    create_mask_from_contour_list(final_mask_pth, path)
                                    dicom_to_png_convert(final_img_pth, path)
                    folder_no += 1

In [102]:
training_path = 'Processed_Dataset/training/'
def rotate_image(img, angle):
    (h, w) = img.shape[:2]
    center = (w / 2, h / 2)
    scale = 1.0
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(img, M, (h, w))
    return rotated
def flip_image(img, flip):
    return cv2.flip(img, flip)

def augment_training_data():
    all_images = os.listdir(training_path + 'images/img')
    for name in all_images:
        im1 = cv2.imread(training_path + 'images/img/' + name)
        im2 = cv2.imread(training_path + 'mask/img/' + name)
        for angle in [45, 90, 135, 180, 225, 270, 315, 0, 1]:
            new_name = name.replace(".png", '') + "_" + str(angle) + ".png"
            if angle < 45:
                image = flip_image(im1, angle)
                mask = flip_image(im2, angle)
            else:
                image = rotate_image(im1, angle)
                mask = rotate_image(im2, angle)
            cv2.imwrite(training_path + 'images/img/' + new_name, image)
            cv2.imwrite(training_path + 'mask/img/' + new_name, mask)


In [114]:
def prepare_validation_data():
    all_images = os.listdir(training_path + 'images/img')
    validation = random.sample(all_images, 500)
    for i in validation:
        shutil.move(training_path + 'images/img/' + i, 'Processed_Dataset/validation/images/img/' + i)
        shutil.move(training_path + 'mask/img/' + i, 'Processed_Dataset/validation/mask/img/' + i)

In [115]:
create_training_testing_data()
augment_training_data()
prepare_validation_data()